In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys

import tensorflow as tf

import yfinance as yf

2025-10-03 19:34:15.326960: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-03 19:34:15.327476: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-03 19:34:15.384329: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-03 19:34:17.119096: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To tur

# Loading the data

Financials

In [2]:
balance_sheet = pd.read_csv('data/financials/AAPL_balance_sheet.csv', header=None)
capital_gains = pd.read_csv('data/financials/AAPL_capital_gains.csv', header=None)
cash_flow = pd.read_csv('data/financials/AAPL_cash_flow.csv', header=None)
income_statement = pd.read_csv('data/financials/AAPL_income_statement.csv', header=None)
ttm_cash_flow = pd.read_csv('data/financials/AAPL_ttm_cash_flow.csv', header=None)

Holders

In [3]:
insider_roster_holders = pd.read_csv('data/holders/AAPL_insider_roster_holders.csv')
institutional_holders = pd.read_csv('data/holders/AAPL_institutional_holders.csv')
mutualfund_holders = pd.read_csv('data/holders/AAPL_mutualfund_holders.csv')
major_holders = pd.read_csv('data/holders/AAPL_major_holders.csv')

Insider

In [4]:
insider_purchases = pd.read_csv('data/insider/AAPL_insider_purchases.csv')
insider_transactions = pd.read_csv('data/insider/AAPL_insider_transactions.csv')

Price

In [5]:
actions = pd.read_csv('data/price/AAPL_actions.csv')
dividends = pd.read_csv('data/price/AAPL_dividends.csv', index_col=0, parse_dates=True)
options = pd.read_csv('data/price/AAPL_options.csv')
ohlc = pd.read_csv('data/price/AAPL_ohlc_and_volume.csv', index_col=0, parse_dates=True)

/tmp/ipykernel_11754/2735305265.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ohlc = pd.read_csv('data/price/AAPL_ohlc_and_volume.csv', index_col=0, parse_dates=True)


# Data exploration

Price

In [6]:
ohlc.tail()

,Close,High,Low,Open,Volume
Price,,,,,
2025-09-29,254.42999267578125,255.0,253.00999450683594,254.55999755859375,40127700
2025-09-30,254.6300048828125,255.9199981689453,253.11000061035156,254.86000061035156,37704300
2025-10-01,255.4499969482422,258.7900085449219,254.92999267578125,255.0399932861328,48713900
2025-10-02,257.1300048828125,258.17999267578125,254.14999389648438,256.5799865722656,42597200
2025-10-03,258.2699890136719,259.239990234375,253.9600067138672,254.6649932861328,28958803


In [7]:
ohlc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11296 entries, Ticker to 2025-10-03
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Close   11295 non-null  object
 1   High    11295 non-null  object
 2   Low     11295 non-null  object
 3   Open    11295 non-null  object
 4   Volume  11295 non-null  object
dtypes: object(5)
memory usage: 529.5+ KB


In [8]:
ohlc.dtypes

Close     object
High      object
Low       object
Open      object
Volume    object
dtype: object

In [9]:
numeric_cols = ['Open', 'High', 'Low', 'Close']

for col in numeric_cols:
    ohlc[col] = pd.to_numeric(ohlc[col], errors='coerce')


In [10]:
ohlc.describe(include='all')

,Close,High,Low,Open,Volume
count,11294.000000,11294.000000,11294.000000,11294.000000,11295
unique,NaN,NaN,NaN,NaN,10731
top,NaN,NaN,NaN,NaN,246400000
freq,NaN,NaN,NaN,NaN,7
mean,26.689622,26.958130,26.394176,26.666813,NaN
std,55.240133,55.781888,54.632918,55.179855,NaN
min,0.037681,0.038109,0.037681,0.038109,NaN
25%,0.245137,0.250706,0.240161,0.245127,NaN
50%,0.466639,0.474725,0.459261,0.466445,NaN
75%,20.927644,21.097475,20.743931,20.965366,NaN


In [11]:
ohlc.memory_usage()

Index     90368
Close     90368
High      90368
Low       90368
Open      90368
Volume    90368
dtype: int64

In [12]:
print(len(ohlc['Close'].unique()))

8281


In [13]:
ohlc['Volume'] = pd.to_numeric(ohlc['Volume'], errors='coerce')
ohlc['High'] = pd.to_numeric(ohlc['High'], errors='coerce')
ohlc['Low'] = pd.to_numeric(ohlc['Low'], errors='coerce')
ohlc['Open'] = pd.to_numeric(ohlc['Open'], errors='coerce')
ohlc['Close'] = pd.to_numeric(ohlc['Close'], errors='coerce')


all_time_low = ohlc['Low'].min()
all_time_low_date = ohlc['Low'].idxmin()
print(f'All time low: {all_time_low} on {all_time_low_date}')

all_time_high = ohlc['High'].max()
all_time_high_date = ohlc['High'].idxmax()
print(f'All time high: {all_time_high} on {all_time_high_date}')

all_time_low_vol = ohlc['Volume'].min()
all_time_low_vol_date = ohlc['Volume'].idxmin()
print(f'All time low volume: {all_time_low_vol} on {all_time_low_vol_date}')

all_time_high_vol = ohlc['Volume'].max()
all_time_high_vol_date = ohlc['Volume'].idxmax()
print(f'All time high volume: {all_time_high_vol} on {all_time_high_vol_date}')

All time low: 0.0376811176538467 on 1982-07-08
All time high: 259.239990234375 on 2025-10-03
All time low volume: 0.0 on 1981-08-10
All time high volume: 7421640800.0 on 2000-09-29


Balance Sheet

In [14]:
balance_sheet.tail()

,0,1,2,3,4
64,Cash Cash Equivalents And Short Term Investments,65171000000.0,61555000000.0,48304000000.0,62639000000.0
65,Other Short Term Investments,35228000000.0,31590000000.0,24658000000.0,27699000000.0
66,Cash And Cash Equivalents,29943000000.0,29965000000.0,23646000000.0,34940000000.0
67,Cash Equivalents,2744000000.0,1606000000.0,5100000000.0,17635000000.0
68,Cash Financial,27199000000.0,28359000000.0,18546000000.0,17305000000.0


# Data preprocessing